In [ ]:
import torch
from torch import nn
from matplotlib import pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D


In [ ]:
class FancyMLP(nn.Module):
    def __init__(self, **kwargs):
        super(FancyMLP, self).__init__(**kwargs)

        self.rand_weight = torch.rand((20, 20), requires_grad=False) # 不可训练参数（常数参数）
        self.linear = nn.Linear(20, 20)

    def forward(self, x):
        x = self.linear(x)
        # 使用创建的常数参数，以及nn.functional中的relu函数和mm函数
        x = nn.functional.relu(torch.mm(x, self.rand_weight.data) + 1)

        # 复用全连接层。等价于两个全连接层共享参数
        x = self.linear(x)
        # 控制流，这里我们需要调用item函数来返回标量进行比较
        while x.norm().item() > 1:
            x /= 2
        if x.norm().item() < 0.8:
            x *= 10
        return x.sum()

In [ ]:
class NestMLP(nn.Module):
    def __init__(self, **kwargs):
        super(NestMLP, self).__init__(**kwargs)
        self.net = nn.Sequential(nn.Linear(40, 30), nn.ReLU()) 

    def forward(self, x):
        return self.net(x)

net = nn.Sequential(NestMLP(), nn.Linear(30, 20), FancyMLP())

net1 = nn.Sequential(nn.Linear(5,5))

x = torch.rand(3,5)

net1(x)


In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.hidden = nn.Linear(3, 2)
        self.act = nn.ReLU()
        self.output = nn.Linear(2, 1)

    def forward(self, x):
        a = self.act(self.hidden(x))
        return self.output(a)

net = MLP()
net.state_dict()

In [ ]:
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer.state_dict()

In [ ]:
torch.cuda.get_device_name(0)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
x = torch.tensor([1,2,3]).to(device)

In [ ]:
x

In [ ]:
import torch
from torch import nn

# 定义一个函数来计算卷积层。它对输入和输出做相应的升维和降维
def comp_conv2d(conv2d, X):
    # (1, 1)代表批量大小和通道数（“多输入通道和多输出通道”一节将介绍）均为1
    X = X.view((1, 1) + X.shape)
    print(X.shape)
    Y = conv2d(X)
    return Y.view(Y.shape[2:])  # 排除不关心的前两维：批量和通道

# 注意这里是两侧分别填充1行或列，所以在两侧一共填充2行或列
conv2d = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding=1)

X = torch.rand(8, 8)
comp_conv2d(conv2d, X).shape

In [ ]:
print((1,1)+(2,2))

In [ ]:
a = torch.tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])

a.shape

In [ ]:
net = nn.Sequential(
        nn.Linear(10, 20),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(20, 15), 
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(15, 10)
        )

x = torch.rand(20, 10)

for name, c in net.named_children():
    x = c(x)
    print(name,"______", x.shape)

        

In [ ]:
a = torch.rand(2,3,3)
a

In [ ]:
a = a.mean(dim=0, keepdim=True).mean(dim=1, keepdim=True)
a

In [ ]:
import time
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..") 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def conv_block(in_channels, out_channels):
    blk = nn.Sequential(nn.BatchNorm2d(in_channels), 
                        nn.ReLU(),
                        nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
    return blk

class DenseBlock(nn.Module):
    def __init__(self, num_convs, in_channels, out_channels):
        super(DenseBlock, self).__init__()
        net = []
        for i in range(num_convs):
            in_c = in_channels + i * out_channels
            net.append(conv_block(in_c, out_channels))
        self.net = nn.ModuleList(net)
        self.out_channels = in_channels + num_convs * out_channels # 计算输出通道数

    def forward(self, X):
        for blk in self.net:
            Y = blk(X)
            X = torch.cat((X, Y), dim=1)  # 在通道维上将输入和输出连结
        return X

In [ ]:
blk = DenseBlock(2,3,8)
x = torch.rand(4,3,8,8)
y = blk(x)
y.shape

In [ ]:
import numpy as np
objp = np.zeros((6*9,3), np.float32)
objp[:,:2] = np.mgrid[0:6,0:9].T.reshape(-1,2)
objp *= 1.2
objp

In [ ]:
import cv2 as cv
img = cv.imread('test.jpg')
cv.imshow('im', img)
cv.waitKey(500)
img = cv.resize(img, (960,480))
print(img.size)
cv.imshow('im', img)
cv.waitKey(500)
cv.destroyAllWindows()

In [ ]:
import numpy as np
objp = np.zeros((1, 6*9, 3), np.float32)
objp[0,:,:2] = np.mgrid[0:6, 0:9].T.reshape(-1, 2)*30
objp

In [ ]:
import re
a = '[sdsad]'
a = re.sub('[\[\]sd]', '', a)
a

In [ ]:
a = "a  b"
print(a.split())


In [ ]:
a = [i for i in range(10)]
a = np.array(a)
a[0]

In [ ]:
a = '1.2'
print(float(a))
a

In [ ]:
a = ['851.19446', '631.96454', '850.45294', '606.37787', '849.7779', '581.1345', '849.17004', '556.3839', '848.73193', '532.3114', '848.3455', '509.43277', '875.5562', '630.39233', '874.6188', '604.2248', '873.6532', '578.63226', '872.7132', '553.72205', '872.0637', '529.46063', '871.3059', '506.33194', '901.02075', '628.0922', '899.38934', '602.03986', '898.2486', '576.4682', '896.7754', '551.0553', '895.75024', '526.63934', '894.55145', '503.14148', '927.0404', '626.06226', '925.35785', '599.6164', '923.547', '573.5644', '921.6574', '548.41016', '920.4528', '523.55316', '918.9476', '499.83658', '953.38477', '623.7945', '951.58124', '596.8947', '949.3174', '570.51416', '947.2819', '545.4884', '945.32825', '520.51636', '943.47253', '496.54892', '980.31006', '621.4369', '977.5882', '594.5057', '975.329', '568.1034', '972.91986', '542.20905', '970.4433', '517.46344', '968.1454', '493.29834', '1007.5476', '618.5754', '1004.6684', '591.5659', '1001.9089', '564.9984', '999.2292', '539.5562', '996.4984', '514.5286', '993.44904', '490.38174', '1034.9849', '615.74475', '1031.6824', '588.5247', '1028.7372', '562.0436', '1025.3925', '536.2487', '1021.7842', '511.08374', '1018.5763', '487.5063', '1062.0992', '612.7502', '1058.5757', '585.5391', '1055.3615', '559.03516', '1051.6493', '533.08014', '1047.5045', '508.42755', '1043.844', '484.31967']
for i in a:
    print(i, float(i))

In [ ]:
import yaml

res = {'a':1, 'b':2}

with open('camera_parameter.yaml') as f:

    data = yaml.load(f, Loader=yaml.FullLoader)

print(data)



In [ ]:
import numpy as np
a = np.arange(5000).reshape(100,50,1)
print(a.shape[::-1])


In [ ]:
a = [[1,2],[3,4]]
b = [[-1,-2]]
import numpy as np
np.array(a) + np.array(b)

In [ ]:
import numpy as np
a = [[[1,2]],[[1,2]],[[1,2]]]
b = np.array(a)
# b.shape
b.shape[::-1]

In [ ]:
b.resize(3,2,1)
b.shape[:2][::-1]

In [ ]:
import math
src = [[650,967],[1209,961],[801,753],[1112,735]]
src = np.array(src)
src =src - np.array([0,540])
src[:, 1] = np.array([540])-src[:, 1]
print(src)
center = (np.average(src[:, 0]), np.average(src[:, 1]))
print('     ', center)

bias = (src - np.array(center))*0.2
for i in range(len(bias)):
    for j in range(len(bias[0])):
        bias[i][j] = math.floor(bias[i][j])

print(bias)

print(bias+src)


In [10]:
import numpy as np
import cv2
import math

img = cv2.imread("./10.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

ret, corners = cv2.findChessboardCorners(gray, (6,9), cv2.CALIB_CB_FAST_CHECK)

if ret:
    print(corners)
    # for i in range(len(corners)):
    #     a = tuple([math.floor(i) for i in corners[i].squeeze()])
    #     cv2.putText(img, str(i), a, cv2.FONT_HERSHEY_SIMPLEX, 0.5,(0,0,0),2,cv2.LINE_AA)
    corners2 = cv2.cornerSubPix(gray,corners, 
                    (11,11), 
                    (-1,-1), 
                    (cv2.TERM_CRITERIA_EPS + 
                    cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01))
    print(corners2)

# cv2.imshow("hah", img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

[[[ 763.3812   684.4941 ]]

 [[ 758.9092   635.788  ]]

 [[ 756.0631   584.9381 ]]

 [[ 754.45026  532.5921 ]]

 [[ 753.59033  480.24738]]

 [[ 754.7303   428.6049 ]]

 [[ 810.4393   687.5122 ]]

 [[ 806.8684   637.64105]]

 [[ 804.4364   585.5024 ]]

 [[ 802.8525   531.98834]]

 [[ 802.29755  478.32098]]

 [[ 803.0064   425.66943]]

 [[ 860.3572   689.5749 ]]

 [[ 857.75336  638.8936 ]]

 [[ 855.4756   585.7337 ]]

 [[ 854.3776   531.38354]]

 [[ 854.023    476.56226]]

 [[ 854.2095   422.5851 ]]

 [[ 912.72675  690.787  ]]

 [[ 911.14374  639.6444 ]]

 [[ 909.64557  586.15375]]

 [[ 908.7993   530.4066 ]]

 [[ 908.34     475.33707]]

 [[ 908.28     420.71335]]

 [[ 965.8736   691.03406]]

 [[ 965.555    639.66113]]

 [[ 965.01984  585.72455]]

 [[ 964.4837   530.02454]]

 [[ 963.8972   474.30927]]

 [[ 963.3789   419.7228 ]]

 [[1019.36554  690.3099 ]]

 [[1019.9725   638.8392 ]]

 [[1020.214    585.03253]]

 [[1020.0308   529.51666]]

 [[1019.5142   474.00192]]

 [[1018.5879   419.3

In [1]:
import cv2
import yaml


a = cv2.imread("./1.jpg")
a = cv2.resize(a,(640,480),interpolation=cv2.INTER_CUBIC)
print(a.shape)
cv2.imshow("s",a)
cv2.waitKey(0)


ModuleNotFoundError: No module named 'yaml'

In [1]:
!nvidia-smi

Tue Mar 22 18:31:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0  On |                  N/A |
| 40%   24C    P8    15W / 130W |    486MiB /  8192MiB |      5%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)


cuda
